In [11]:
import numpy as np
import pandas as pd
import random
import time
from sklearn.model_selection import RepeatedKFold
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from scipy import linalg
from sklearn.neural_network import MLPClassifier

In [12]:
def ASBM(n,k):
    A = np.zeros((n,n))
    #np.random.seed(1)
    for i in range(0,n):
        for j in range(0,n):
            if y[i] == y[j]:
                A[i,j] = np.random.binomial(1, 0.13)
            else:
                A[i,j] = np.random.binomial(1, 0.1)
    for i in range(0,n):
        for j in range(i,n):
            A[i,j]=A[j,i]
    return A

In [13]:
def ADCSBM(n,k):
    A = np.zeros((n,n))
    theta = np.random.beta(1, 4, n)
    theta2 = theta/sum(theta)
    B=np.array([[0.9,0.1,0.1],[0.1,0.6,0.1],[0.1,0.1,0.3]])
    for i in range(0,n):
        for j in range(i+1,n):
            if y[i] == y[j] == 1:
                A[i,j]=np.random.binomial(1,theta[i]*theta[j]*B[0,0])
            elif y[i] == y[j] == 2:
                A[i,j]=np.random.binomial(1,theta[i]*theta[j]*B[1,1])
            elif y[i] == y[j] == 3:
                A[i,j]=np.random.binomial(1,theta[i]*theta[j]*B[2,2])
            else:
                A[i,j]=np.random.binomial(1,theta[i]*theta[j]*0.1)
    for i in range(0,n):
        for j in range(0,i):
            A[i,j]=A[j,i]
    return A

In [14]:
def ARDPG(n,k):
    A = np.zeros((n,n))
    X = np.random.uniform(0.0, 1.0, (n,1))
    for i in range(0,n):
        if y[i] == 1:
            X[i,:] = np.random.beta(1,3)
        elif y[i] == 2:
            X[i,:] = np.random.beta(3,3)
        else:
            X[i,:] = np.random.beta(3,1)
    P=np.matmul(X,X.transpose())
    for i in range(0,n):
        for j in range(1,n):
            A[i,j]=np.random.binomial(2,P[i,j])
    for i in range(0,n):
        for j in range(0,i):
            A[i,j] = A[j,i]
            A[i,i]=0
    return A

In [15]:
def y(n):
    y = np.zeros((n,1))
    #np.random.seed(2)
    for i in range(0,n):
        y[i,0] = np.random.choice(np.arange(1, 4), p=[0.2, 0.3,0.5])
    return(y)

In [16]:
def EncoderLDA(A):
#Split Data
    rkf = RepeatedKFold(n_splits=5, n_repeats=10, random_state=5)
    for train_index, test_index in rkf.split(A):
         #print("TRAIN:", train_index, "TEST:", test_index)
         X_train, X_test = A[train_index], A[test_index]
         y_train, y_test = y[train_index], y[test_index]
         X1 = X_train.transpose()
         X_train, X_test = X1[train_index], X1[test_index]
#ASE*LDA
    begin1 = time.time()
    U, s, VT = np.linalg.svd(A)
    newx=([])
    d=2
    newx=np.concatenate(U[:,0:d]*s[0:d]**0.5)
    newx=np.reshape(newx,(n,2))
    clf2 = LinearDiscriminantAnalysis()
    #print(np.shape(newx))
    newx_train, newx_test =newx[train_index], newx[test_index]
    clf2.fit(newx_train, y_train.ravel())
    ASELDAACC=clf2.score(newx_test, y_test.ravel())
    time.sleep(2) 
    end1 = time.time()
    ASELDATIME=end1-begin1
    eigenvalues=s**2
    Top=eigenvalues[:10]
#nk,w,Z
    begin5 = time.time()
    global k
    nk = np.zeros((1,k))
    for i in range(0,len(y_train)-1):
        nk[0,int(y_train[i,0]-1)]=nk[0,int(y_train[i,0]-1)]+1
    w = np.zeros((int(np.size(y_train)),k))
    for i in range(0,int(np.size(y_train))):
        k=int(y_train[i])
        w[i][k-1]=1/nk[0,k-1]
    z_train=np.matmul(X_train,w)
    z_test = np.matmul(X_test,w)
    time.sleep(5)
    end5 = time.time()
    zTIME=end5 - begin5
#AEE*MLP
    begin2 = time.time()
    clf3 = MLPClassifier(random_state=2, max_iter=5000)
    clf3.fit(z_train, y_train.ravel())
    AEEMLPACC = clf3.score(z_test, y_test)
    time.sleep(3) 
    end2 = time.time()
    AEEMLPTIME=end2-begin2
#AEE*LDA
    begin3 = time.time()
    clf4 = LinearDiscriminantAnalysis()
    clf4.fit(z_train,y_train.ravel())
    AEELDAACC = clf4.score(z_test, y_test.ravel())
    time.sleep(4) 
    end3 = time.time()
    AEELDATIME=end3-begin3
#Get result
    Acc=np.array([ASELDAACC,AEEMLPACC,AEELDAACC])
    Time=np.array([ASELDATIME,AEEMLPTIME,AEELDATIME])
    print(f"Accuracy for each method: {Acc}")
    print(f"Time for each method: {Time}")

In [17]:
n=1000
k=3
y=y(n=n)

In [18]:
EncoderLDA(ASBM(n=n,k=k))

(1000, 2)
Accuracy for each method: [0.66  0.795 0.81 ]
Time for each method: [2.52763295 5.01148391 4.00483465]


In [19]:
#EncoderLDA(ADCSBM(n=n,k=k))

In [20]:
#EncoderLDA(ARDPG(n=n,k=k))